In [1]:
import pandas as pd

# 指定文件路径
file_path = '/Users/laihuiqian/Documents/weibo_0925/total1-0925.csv'

# 读取.csv文件到DataFrame
df = pd.read_csv(file_path, encoding='utf-8')

# 提取“微博正文”这一列
weibo_content = df["微博正文"]

# 打印“微博正文”
print(weibo_content)


0        【2022·年度爱用物🏆彩妆篇】超长25min完整版！20大品类！100+件爱用精选！一路常...
1        《云之羽》原班造型团队🎬      🌟#虞书欣云为衫妆造还原#㊙️虞书欣本剧御用化妆师@侍慧...
2        谁懂！！😭欣欣把云为衫的剧组衣服借我穿了！！！#虞书欣云为衫妆造还原##微博变美手册# #仿...
3            回家啦！饭后散步中~特别舒服的初秋上海！💨🍂#一枝南南[超话]#  [组图共6张] 原图 
4        9月第一天！！水逆退散！！来接好运🍀🎁————固定栏目「pr礼物」上线～评论区许愿池开放✌🏻...
                               ...                        
13000    𝙉𝙞𝙘𝙚 𝙩𝙤 𝘾 𝙮𝙤𝙪现场做实验的一天🔅#美妆生活##motd##好物分享#  [组图共...
13001    𝙄𝙉𝙏𝙊𝙉𝙀 𝙀𝘿𝙄𝙏𝙄𝙊𝙉_心慕与你色彩编辑部实习编辑上线💗#2023的她们##动静皆风尚...
13002    最近天气太好啦～到处都是春日的信号🌿☁️#2023的她们##美妆生活##动静皆风尚#  [组...
13003    啾咪！ʙʟᴜᴇ ʙʟᴜᴇ皮闪现💙今天打卡了科颜氏「超CHILL」保湿街区！超多的沉浸式体验场...
13004    《关于相册里的囤图这件事》✎ 迟到的➌月ᴘʟᴏɢ请查收 ✓#2023的她们##日常碎片plo...
Name: 微博正文, Length: 13005, dtype: object


In [6]:
import lda
import pandas as pd
import jieba

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn

def open_dict(Dict):
    path = '%s.txt' % Dict
    dictionary = open(path, 'r', encoding='utf-8')
    dict = []
    for word in dictionary:
        word = word.strip('\n')
        dict.append(word)
    return dict

clearwords=open_dict('clearwords')

#分词
def chinese_word_cut(mytext):
    tempcut=jieba.cut(str(mytext))
    return " ".join(set(tempcut)-set(clearwords))

#打印前n_top_words关键词
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

# df = pd.read_excel("metoo20181221.xlsx",'Sheet1',index_col=None,na_values=['NA'])
# df.shape
df["content_cutted"] = df["微博正文"].apply(chinese_word_cut)
n_features = 5000

tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(df.content_cutted)
vocab=tf_vectorizer.get_feature_names_out()
model = lda.LDA(n_topics=17, n_iter=100, random_state=1)  
model.fit(tf)
print('model done')

#主题-单词（topic-word）分布
topic_word = model.topic_word_ 
print("shape: {}".format(topic_word.shape))
print(vocab[:3])
print(topic_word[:, :3])
for n in range(12):
    sum_pr = sum(topic_word[n,:])  
    print("topic: {} sum: {}".format(n, sum_pr))

#计算各主题Top-N个单词
import numpy as np
n = 20
for i, topic_dist in enumerate(topic_word):  
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n+1):-1]  
    print('*Topic {}\n- {}'.format(i, ' '.join(topic_words)))  

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
INFO:lda:n_documents: 13005
INFO:lda:vocab_size: 5000
INFO:lda:n_words: 298022
INFO:lda:n_topics: 17
INFO:lda:n_iter: 100
INFO:lda:<0> log likelihood: -3646656
INFO:lda:<10> log likelihood: -2656670
INFO:lda:<20> log likelihood: -2514413
INFO:lda:<30> log likelihood: -2461563
INFO:lda:<40> log likelihood: -2437192
INFO:lda:<50> log likelihood: -2424579
INFO:lda:<60> log likelihood: -2415868
INFO:lda:<70> log likelihood: -2408600
INFO:lda:<80> log likelihood: -2403769
INFO:lda:<90> log likelihood: -2398470
INFO:lda:<99> log likelihood: -2397472


model done
shape: (17, 5000)
['00' '01' '02']
[[4.61169526e-07 4.61169526e-07 4.61169526e-07]
 [9.30578820e-07 9.30578820e-07 9.30578820e-07]
 [6.23519142e-07 6.23519142e-07 6.23519142e-07]
 [5.79619626e-03 5.63245555e-04 5.62682872e-07]
 [5.73789305e-07 5.74363094e-04 1.03339454e-03]
 [8.65276456e-07 8.65276456e-07 8.65276456e-07]
 [5.98909984e-07 5.98909984e-07 5.98909984e-07]
 [4.25387102e-07 4.25387102e-07 4.25387102e-07]
 [1.05418511e-06 1.05418511e-06 1.05418511e-06]
 [5.16368894e-07 5.16368894e-07 5.16368894e-07]
 [6.83854202e-07 6.83854202e-07 3.42610955e-04]
 [5.21975154e-07 5.21975154e-07 5.21975154e-07]
 [3.20728696e-07 3.20728696e-07 3.20728696e-07]
 [4.31685733e-07 4.31685733e-07 4.31685733e-07]
 [6.36294222e-07 6.36294222e-07 6.36294222e-07]
 [6.03427468e-07 6.03427468e-07 6.03427468e-07]
 [1.05875714e-03 7.05367849e-07 7.05367849e-07]]
topic: 0 sum: 0.9999999999999561
topic: 1 sum: 0.9999999999999183
topic: 2 sum: 0.9999999999999251
topic: 3 sum: 1.0000000000000449
topic